## Exercise 1

In [1]:
from textblob import TextBlob

with open('burbank.txt', 'r') as file:
    text_summary = file.read()

blob = TextBlob(text_summary)
sentiment = blob.sentiment

print("Sentiment Polarity:", sentiment.polarity)
print("Sentiment Subjectivity:", sentiment.subjectivity)


Sentiment Polarity: 0.09869334480780263
Sentiment Subjectivity: 0.3790877796901893


## Exercise 2

In [4]:
blob_representative = TextBlob(text_summary)
sentiment_representative = blob_representative.sentiment

print("sentiment representative polarity ",sentiment_representative.polarity)
print("sentiment representative subjectivity ",sentiment_representative.subjectivity)

sentiment representative polarity  0.09869334480780263
sentiment representative subjectivity  0.3790877796901893


## Exercise 3

In [19]:
from textblob import TextBlob
from textblob import Word
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
# Read text from file
with open('burbank.txt', 'r') as file:
    text = file.read()

# Create a TextBlob object
blob = TextBlob(text)

# Tokenize the text into words
words = blob.words

# Create a frequency dictionary to store the count of each word
word_freq = {}

# Iterate over each word and count its frequency
for word in words:
    if word not in word_freq:
        word_freq[word] = 1
    else:
        word_freq[word] += 1

# Sort the words by frequency
sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

# Print the top 10 most frequent words and their frequencies
print("Top 10 Most Frequent Words:")
for word, freq in sorted_words[:10]:
    print(f"{word}: {freq}")
    

Top 10 Most Frequent Words:
the: 100
of: 50
to: 39
and: 36
that: 20
in: 19
a: 16
The: 14
FAA: 13
Burbank: 12
Number of stopwords: 326
Example stopwords: ['which', 'rather', '‘ll', 'nor', 'doing', 'go', 'out', 'together', 'were', "'ve"]


## Exercise 4

In [28]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string

# Load datasets
amazon_df = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
imdb_df = pd.read_csv('imdb_labelled.txt', sep='\t', header=None)
yelp_df = pd.read_csv('yelp_labelled.txt', sep='\t', header=None)

# Combine datasets
amazon_df.columns = ['Review', 'Label']
imdb_df.columns = ['Review', 'Label']
yelp_df.columns = ['Review', 'Label']

amazon_df['Company'] = 'Amazon'
imdb_df['Company'] = 'IMDb'
yelp_df['Company'] = 'Yelp'

comb_data = pd.concat([amazon_df, imdb_df, yelp_df], ignore_index=True)

# Explore structure of comb_data
print(comb_data.head())
print(comb_data.info())

# Save combined dataset
comb_data.to_csv('Sentiment_Analysis_Dataset.csv', index=False)

# Check for null values
print(comb_data.isnull().sum())

# Define stopwords
stopwords = list(STOP_WORDS)

# Define punctuations
punctuations = string.punctuation

# Load English language model for spaCy
nlp = spacy.load('en_core_web_sm')

# Tokenization function
def tokenize(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower().strip() for token in doc if token.lemma_ != "-PRON-"]
    tokens = [token for token in tokens if token not in stopwords and token not in punctuations]
    return tokens

# Import necessary functions and classes
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

# Custom transformer class
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean text
def clean_text(text):
    return text.strip().lower()

# Vectorize and use LinearSVC as a classifier
vectorizer = CountVectorizer(tokenizer=tokenize, ngram_range=(1,1))
classifier = LinearSVC()

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(comb_data['Review'], comb_data['Label'], test_size=0.2, random_state=42)

# Create pipeline
pipe_countvect = Pipeline([('cleaner', predictors()),
                           ('vectorizer', vectorizer),
                           ('classifier', classifier)])

# Fit the data
pipe_countvect.fit(X_train, y_train)

# Predict with the test dataset
y_pred = pipe_countvect.predict(X_test)

# Determine accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Sample prediction
sample_text = "This movie was amazing!"
sample_pred = pipe_countvect.predict([sample_text])[0]
print(sample_text, "Prediction:", sample_pred)


                                              Review  Label Company
0  So there is no way for me to plug it in here i...      0  Amazon
1                        Good case, Excellent value.      1  Amazon
2                             Great for the jawbone.      1  Amazon
3  Tied to charger for conversations lasting more...      0  Amazon
4                                  The mic is great.      1  Amazon
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Review   2748 non-null   object
 1   Label    2748 non-null   int64 
 2   Company  2748 non-null   object
dtypes: int64(1), object(2)
memory usage: 64.5+ KB
None
Review     0
Label      0
Company    0
dtype: int64


/Users/kiran_g/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/kiran_g/Library/Python/3.9/lib/python/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 0.8054545454545454
This movie was amazing! Prediction: 1
